<a href="https://colab.research.google.com/github/Stefanosdl/Neural_Networks_and_clustering_of_images/blob/main/Assignment3_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 2.x
import tensorflow as tf
# print("Tensorflow version " + tf.__version__)

try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
#   print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

INFO:tensorflow:Initializing the TPU system: grpc://10.46.26.218:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.46.26.218:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [2]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
#Datasets
path = "/content/drive/My Drive/Project_Ergasia3/Datasets/"
inputfile = path + "t10k-images.idx3-ubyte"

In [4]:
import re
import numpy as np
import pandas as pd 
from matplotlib import pyplot as plt

In [5]:
def reader(dataset):
    # create a dictionary of lists to store all images
    all_images = {}
    all_images[0] = []
    with open(dataset, "rb") as f:
        counter = 0
        # read metadata
        byte = f.read(4)
        while byte:
            if counter == 0:
                magic_number = int.from_bytes(byte, "big")
            elif counter == 1:
                number_of_images = int.from_bytes(byte, "big")
            elif counter == 2:
                rows = int.from_bytes(byte, "big")
            elif counter == 3:
                cols = int.from_bytes(byte, "big")
                break
            counter += 1
            byte = f.read(4)
        # start reading the images 
        byte_counter = 0
        image_counter = 0
        dimensions = rows * cols
        print (magic_number, " ", number_of_images," ", rows," ", cols, " ", dimensions)
        # read images, byte byte
        byte = f.read(1)
        while byte:
            # store byte in the 
            all_images[image_counter].append(int.from_bytes(byte, "big"))
            byte_counter += 1
            if (byte_counter == dimensions):
                # next image
                image_counter += 1
                byte_counter = 0
                # initialize the list for this image
                all_images[image_counter] = []
            byte = f.read(1)
    # finished with reading of file
    # remove last item number_of_images index that is anyway an empty list
    all_images.popitem()
    print ("finished reading from file")
    # return a tuple of number of images, dimensions and all_images dict
    return all_images

def reader_labels(dataset):
    # create a dictionary of lists to store all images
    all_labels = []
    with open(dataset, "rb") as f:
        counter = 0
        # read metadata
        byte = f.read(4)
        while byte:
            if counter == 0:
                magic_number = int.from_bytes(byte, "big")
            elif counter == 1:
                number_of_items = int.from_bytes(byte, "big")
                break
            counter += 1
            byte = f.read(4)

		# start reading the labels 
        byte = f.read(1)
        while byte:
			# store byte in the 
            all_labels.append(int.from_bytes(byte, "big"))
            byte = f.read(1)
            # finished with reading of file
    return all_labels

def read_all_files(training_set, training_labels, test_set, test_labels):
	train_images = reader(training_set)
	train_labels = reader_labels(training_labels) 
	test_images = reader(test_set)
	test_labels = reader_labels(test_labels)
	# labels are lists of 10000 items
	return (train_images, train_labels, test_images, test_labels)


In [6]:
import re
import numpy as np
import pandas as pd 
from matplotlib import pyplot as plt
import tensorflow as tf
import keras
from keras import layers
from keras import backend as K
from keras.layers.embeddings import Embedding
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split

In [7]:
def convolutionalAutoencoder():
    keras.backend.clear_session()
    print("[INFO] building autoencoder...")
    model = tf.keras.models.Sequential()
    model.add(layers.Input(shape=(28, 28, 1)))
    model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same', strides=2))
    model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same', strides=2))
    model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same', strides=3))

    model.add(layers.Flatten())
    model.add(layers.Dense(10, activation='relu'))
    # model.add(layers.Dropout(0.3))
    model.add(layers.Dense(1152, activation='relu'))

    model.add(layers.Reshape((3, 3, 128)))

    model.add(layers.Conv2DTranspose(64, kernel_size=3, strides=2, activation='relu'))
    model.add(layers.Conv2DTranspose(32, kernel_size=3, strides=2, activation='relu', padding='same'))
    model.add(layers.Conv2DTranspose(1, kernel_size=3, strides=2, activation='sigmoid', padding='same'))

    return model


In [8]:
# training 
def training(autoencoder, all_images, epochs=10, batch_size=128):
	print("[INFO] training started...")
	df = pd.DataFrame.from_dict(all_images, orient='index')
	# now in df we have a dataframe with size: dimensions x number_of_images with all of our images
	opt = keras.optimizers.Adam(lr=1e-3)
	# loss = 'binary_crossentropy'
	autoencoder.compile(loss="mse", optimizer=opt, metrics = ['accuracy'])

	# To train it we will need the data
	train, test = train_test_split(df, test_size=0.33, random_state=42)

	train = train.astype('float32') / 255.
	test = test.astype('float32') / 255.

	train = train.to_numpy()
	test = test.to_numpy()

	train = np.reshape(train, (len(train), 28, 28, 1))
	test = np.reshape(test, (len(test), 28, 28, 1))

	history = autoencoder.fit(train, train,
					epochs=epochs,
					batch_size=batch_size,
					shuffle=True,
					validation_data=(test, test),
					callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss', patience=30, restore_best_weights=True)])

	return (history, test)

In [9]:
# run this
epochs = 20
autoencoder = convolutionalAutoencoder()
all_images = reader(inputfile)
(history, test) = training(autoencoder, all_images)
autoencoder.summary()

[INFO] building autoencoder...
2051   10000   28   28   784
finished reading from file
[INFO] training started...
Epoch 1/10
53/53 [==============================] - 3s 40ms/step - loss: 0.1945 - accuracy: 0.7773 - val_loss: 0.1104 - val_accuracy: 0.8052
Epoch 2/10
53/53 [==============================] - 1s 27ms/step - loss: 0.1077 - accuracy: 0.8038 - val_loss: 0.1046 - val_accuracy: 0.8034
Epoch 3/10
53/53 [==============================] - 2s 31ms/step - loss: 0.1016 - accuracy: 0.8019 - val_loss: 0.0676 - val_accuracy: 0.7962
Epoch 4/10
53/53 [==============================] - 1s 28ms/step - loss: 0.0645 - accuracy: 0.8006 - val_loss: 0.0574 - val_accuracy: 0.7942
Epoch 5/10
53/53 [==============================] - 1s 27ms/step - loss: 0.0544 - accuracy: 0.7917 - val_loss: 0.0487 - val_accuracy: 0.7880
Epoch 6/10
53/53 [==============================] - 1s 27ms/step - loss: 0.0465 - accuracy: 0.7900 - val_loss: 0.0406 - val_accuracy: 0.7945
Epoch 7/10
53/53 [======================

In [10]:
encoder = tf.keras.models.Sequential()
encoder.add(layers.Input(shape=(28, 28, 1)))
for i in range(0, len(autoencoder.layers)//2):
    encoder.add(autoencoder.layers[i])
decoded_imgs=encoder.predict(test)
print(encoder.summary())
print(decoded_imgs.max())
print(decoded_imgs.min())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 14, 14, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 7, 7, 64)          18496     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 128)         73856     
_________________________________________________________________
flatten (Flatten)            (None, 1152)              0         
_________________________________________________________________
dense (Dense)                (None, 10)                11530     
Total params: 104,202
Trainable params: 104,202
Non-trainable params: 0
_________________________________________________________________
None
88.93832
0.0


In [11]:
# def normalize(x):
#   return ((25500*(x - np.min(x))/np.ptp(x)).astype(int))

normalized = []
for i in range(len(decoded_imgs)):
    normalized.append((25500*(decoded_imgs[i] - np.min(decoded_imgs[i]))/np.ptp(decoded_imgs[i])).astype(int))

normalized = np.asarray(normalized)
# normalized = dict(enumerate(normalized, 1)) 

In [12]:
# TODO: CHANGE THE PATH!!!!!!!!
# outputfile = open("decoded_images", "wb")

# for key, value in normalized.items():
#     # for pic_value in value:
#     outputfile.write(bytearray(value))
# outputfile.close()

In [13]:
import struct
normalized.astype(np.ushort)
# TODO: CHANGE THE PATH
outputfile = open("images_new_space","wb")
number_of_images = normalized.shape[0]
dimensions = normalized.shape[1]
# write metadata
# enforce big endian
myfmt = '>i'
# write magic number
bin = struct.pack(myfmt, 2081)
outputfile.write(bin)
# write num of images
bin = struct.pack(myfmt, number_of_images)
outputfile.write(bin)
# write num of rows (1)
bin = struct.pack(myfmt, 1)
outputfile.write(bin)
# write dimensions
bin = struct.pack(myfmt, dimensions)
outputfile.write(bin)
# enforce little endian
myfmt = 'H'*dimensions
myfmt = '<' + myfmt
#  You can use 'H' for unsigned short and <, for little endian to force endinness
for i in range(0, number_of_images):
    arr = np.array(normalized[i])
    bin = struct.pack(myfmt, *arr)
    outputfile.write(bin)

outputfile.close()


In [14]:
import os
os.path.getsize('images_new_space')

66016